# **Virtual Home Communitication Guide**

## **1. Import community and establish connection**  

This section initializes a connection to VirtualHome by specifying the executable file's path and setting up communication. The `UnityCommunication` class from `comm_unity` allows configuration of the host IP and port, defaulting to `localhost:8080`.

### **Code Explanation**  
- **Import required libraries**:  
  - `pickle`, `pprint`: For data serialization and pretty-printing.  
  - `comm_unity` from `virtualhome.simulation.unity_simulator`: Handles communication with the VirtualHome environment.  
  - `PIL.Image`: For image processing.  
  - `time`, `numpy`: Utility modules for time management and numerical operations.  

- **Initialize UnityCommunication**:  
  - Set `YOUR_FILE_NAME` to the path of the VirtualHome executable.  
  - Create a `UnityCommunication` instance with the executable file path.  


In [2]:
import pickle
import pprint

from virtualhome.simulation.unity_simulator import comm_unity
from PIL import Image
import time
import numpy as np

# init UnityCommunication
YOUR_FILE_NAME = "D:\\programs\\windows_exec.v2.2.4\\VirtualHome.exe"
comm = comm_unity.UnityCommunication(file_name=YOUR_FILE_NAME)


['D:\\programs\\windows_exec.v2.2.4\\VirtualHome.exe', '-batchmode', '-http-port=8080', '-logFile D:\\code\\llm-reinforce-learning\\docs\\tutorial/Player_8080.log']
Getting connection...


## **2. Reset the Environment**  

The `comm.reset()` method resets the VirtualHome simulation environment by clearing characters and scene modifications before reloading the selected scene.  

### **Code Explanation**  
- **Function Definition** (`reset(self, environment=None)`):  
  - **Resets the scene**: Removes all characters and scene changes.  
  - **Loads a new environment**: If an `environment` index (0–49) is provided, it loads the specified apartment.  
  - **Sends a reset request**: Uses `post_command` to communicate with the VirtualHome simulator.  
  - **Returns a success flag** (`True` or `False`), indicating whether the reset was successful.  

### **Confirm Code Implementation of Commnicator**  

```python
def reset(self, environment=None):
    """
    Reset scene. Deletes characters and scene changes, and loads the scene in scene_index

    :param int environment: integer between 0 and 49, corresponding to the apartment we want to load
    :return: success (bool)
    """
    response = self.post_command({'id': str(time.time()), 'action': 'reset',
                                    'intParams': [] if environment is None else [environment]})
    return response['success']

# Reset the environment and load apartment 0
x = comm.reset(0)
print(f'Reset: {x}')
```  

This ensures that the simulation starts from a clean state before executing further interactions.

In [3]:
x = comm.reset(0)
print(f'reset: {x}')

reset: True


## **3. Environment Graph**  

The `environment_graph()` method retrieves a structured representation of the current environment, including object locations, attributes, and relationships.  

### **Code Explanation**  
- **Retrieve the environment graph**:  
  - `_, g = comm.environment_graph()` gets the graph representation, where `g` contains nodes (objects) and edges (relationships).  

- **Identify specific objects**:  
  - `salmon_id`: Extracts the unique ID of the object labeled `"salmon"`.  
  - `microwave_id`: Extracts the unique ID of the `"microwave"`.  

- **Check relationships**:  
  - Searches for an edge (relationship) where the `"salmon"` is inside the `"microwave"`.  
  - The relationship data structure contains `from_id` (salmon) and `to_id` (microwave).  


In [4]:
_, g = comm.environment_graph()
salmon_id = [node['id'] for node in g['nodes'] if node['class_name'] == 'salmon'][0]
microwave_id = [node['id'] for node in g['nodes'] if node['class_name'] == 'microwave'][0]

salmon_in_microwave = [
    relation for relation in g['edges']
    if
    relation['from_id'] == salmon_id and relation['to_id'] == microwave_id
]
print("Before script execution: Salmon in microwave =", salmon_in_microwave)


Before script execution: Salmon in microwave = [{'from_id': 328, 'to_id': 314, 'relation_type': 'ON'}]


## **4. Characters**  

VirtualHome allows adding one or more characters to the environment. The `comm.add_character()` method is used to introduce a character into the simulation. 
This command places a character into the VirtualHome scene, enabling interactions and actions within the simulation.

### **Agent List**  
A list of available agent models can be found at:  
[VirtualHome Agents](http://virtual-home.org/documentation/master/kb/agents.html)  

### **Code Explanation**  
- **Adding a character**:  
  - The function `comm.add_character('Chars/Male1')` adds a male character model (`Male1`) to the scene.  
  - Different character models can be used based on the available agents list.  


In [5]:
comm.add_character('Chars/Male1')

True

## **5. Script Execution and Rendering**  

The `comm.render_script()` method is used to control the execution and rendering of commands within the simulation. It allows for the animation of actions in the form of images or videos, and you can configure various parameters to control the rendering behavior.  
This method executes the series of commands and returns a result indicating the success or failure of the script execution, without rendering the animation.

### **Code Explanation**  
- **Script Creation**:  
  A list of actions (script) is defined, where each entry represents a command for the character (`char0`):
  - Walk to the object (`salmon`).
  - Grab the `salmon`.
  - Open the `microwave`.
  - Put the `salmon` into the `microwave`.
  - Close the `microwave`.  

  The IDs of the `salmon` and `microwave` are dynamically inserted into the script commands.  

- **Rendering Options**:  
  - `recording=False`: Disable video recording of the rendered actions.  
  - `skip_animation=True`: Skip the actual animation rendering, focusing on script execution without visual animation.  



In [6]:
script = [
    '<char0> [walk] <salmon> ({})'.format(salmon_id),
    '<char0> [grab] <salmon> ({})'.format(salmon_id),
    '<char0> [open] <microwave> ({})'.format(microwave_id),
    '<char0> [putin] <salmon> ({}) <microwave> ({})'.format(salmon_id, microwave_id),
    '<char0> [close] <microwave> ({})'.format(microwave_id)
]
# r = comm.render_script(script, recording=True, frame_rate=10)
r = comm.render_script(script, recording=False, skip_animation=True)
print("rendered script =", r)

rendered script = (True, {'0': {'message': 'Success'}})


## **6. Check the Updated Status**  

After executing the script, the environment is checked again to verify that the `salmon` has been successfully placed inside the `microwave`. The state graph is updated, reflecting the new relationships between objects.  
This block of code verifies the successful execution of the script by checking if the salmon was indeed placed inside the microwave and prints the updated state of the environment.

### **Code Explanation**  
- **Retrieve Updated Environment Graph**:  
  - `_, g = comm.environment_graph()` fetches the updated graph of the environment after executing the script.  

- **Verify the Relationship**:  
  - The `salmon_in_microwave` list is created by searching the edges (relationships) for one where the `salmon` is inside the `microwave`. This confirms the action was successful.  

- **Print the Environment Information**:  
  - `pprint.pprint(g['nodes'][:5])`: Prints the first 5 nodes (objects) in the environment, showing their details.  
  - `pprint.pprint(g['edges'][:5])`: Prints the first 5 edges (relationships) in the environment, illustrating how objects are connected.  


In [7]:
_, g = comm.environment_graph()


salmon_in_microwave = [
    relation for relation in g['edges']
    if
    relation['from_id'] == salmon_id and relation['to_id'] == microwave_id
]
print("After script execution: Salmon in microwave =", salmon_in_microwave)


pprint.pprint(g['nodes'][:5])
pprint.pprint(g['edges'][:5])


After script execution: Salmon in microwave = [{'from_id': 328, 'to_id': 314, 'relation_type': 'INSIDE'}]
[{'bounding_box': {'center': [-2.50765753, 0.8730474, -3.26845956],
                   'size': [1.34859729, 1.86971879, 0.515388]},
  'category': 'Characters',
  'class_name': 'character',
  'id': 1,
  'obj_transform': {'position': [-9.563976, 1.25, -3.541883],
                    'rotation': [0.0, 0.987328768, 0.0, 0.158688009],
                    'scale': [1.0, 1.0, 1.0]},
  'prefab_name': 'Male1',
  'properties': [],
  'states': []},
 {'bounding_box': {'center': [-5.135, 1.247, 0.723], 'size': [8.0, 3.0, 5.5]},
  'category': 'Rooms',
  'class_name': 'bathroom',
  'id': 11,
  'obj_transform': {'position': [-6.385, -0.003, -0.527],
                    'rotation': [0.0, 0.0, 0.0, 1.0],
                    'scale': [1.0, 1.0, 1.0]},
  'prefab_name': 'PRE_ROO_Bathroom_01',
  'properties': [],
  'states': []},
 {'bounding_box': {'center': [-7.635, -0.003, -0.52699995],
              

## **7. Predefined Objects**  

In VirtualHome, various predefined object categories are available, including rooms, furniture, and food items. The script allows for checking whether certain objects exist in the current environment and what relationships they have.  

### **Code Explanation**  
- **Predefined Lists**:  
  - `food_list`: A list of different food items available in the environment.  
  - `room_list`: A list of rooms, such as the `kitchen`, `livingroom`, etc.  
  - `object_list`: A list of furniture and objects, such as `microwave`, `coffeetable`, etc.  

- **Helper Functions**:  
  - `where_obj(xid: int, g: Any)`: Finds the name of the object by its ID (`xid`) from the graph (`g`).  
  - `find_obj(objs: list[str], g: Any)`: Searches for the objects in the `objs` list and prints out their ID and relationships within the environment. It searches for nodes matching object types and then checks the relationships involving those objects.  


In [8]:
from typing import Any
food_list = ['salmon', 'apple', 'bananas', 'pancake', 'peach', 'pear', 'pie', 'potato',
             'salad', 'tomato', 'wine', 'beer', 'plum', 'orange', 'milkshake', 'mincedmeat',
             'lemon', 'juice', 'chocolatesyrup', 'chicken', 'carrot']

room_list = ['kitchen', 'livingroom', 'kitchen', 'bedroom']
object_list = ['microwave', 'coffeetable', 'kitchentable', 'wallshelf', 'kitchencounter', 'desk', 'fridge', 'bookshelf', 'stove']


def where_obj(xid: int, g: Any):
    return [node['class_name'] for node in g['nodes'] if node['id'] == xid]


def find_obj(objs: list[str], g: Any):
    for obj in objs:
        xids = [node['id'] for node in g['nodes'] if node['class_name'] == obj]
        if len(xids) <= 0:
            continue

        rels = [f'{edge['relation_type']}-{where_obj(edge['to_id'], g)}' for edge in g['edges'] if
                edge['from_id'] == xids[0]]
        print(f'obj_name: {obj}, id: {xids[0]}')

        print(f'relations: {rels}')


In [9]:
res = comm.reset(0)
# comm.add_character('Chars/Male1')
print('res: ', res)
res, g = comm.environment_graph()
find_obj(food_list, g)

res:  True
obj_name: salmon, id: 328
relations: ["INSIDE-['kitchen']", "ON-['microwave']"]
obj_name: apple, id: 438
relations: ["INSIDE-['livingroom']", "ON-['coffeetable']", "FACING-['computer']"]
obj_name: bananas, id: 316
relations: ["INSIDE-['kitchen']", "INSIDE-['bookshelf']", "FACING-['tv']"]
obj_name: peach, id: 442
relations: ["INSIDE-['livingroom']", "ON-['coffeetable']", "FACING-['computer']"]
obj_name: pie, id: 320
relations: ["INSIDE-['kitchen']", "ON-['kitchentable']", "ON-['rug']", "FACING-['tv']", "FACING-['clock']"]
obj_name: plum, id: 444
relations: ["INSIDE-['livingroom']", "ON-['coffeetable']", "FACING-['computer']"]
obj_name: chocolatesyrup, id: 332
relations: ["INSIDE-['kitchen']", "ON-['wallshelf']"]


In [10]:
find_obj(object_list, g)

obj_name: microwave, id: 314
relations: ["INSIDE-['kitchen']", "ON-['kitchencounter']", "ON-['kitchencounterdrawer']"]
obj_name: coffeetable, id: 113
relations: ["INSIDE-['bedroom']", "ON-['floor']", "FACING-['computer']"]
obj_name: kitchentable, id: 231
relations: ["INSIDE-['kitchen']", "FACING-['tv']"]
obj_name: wallshelf, id: 44
relations: ["INSIDE-['bathroom']"]
obj_name: kitchencounter, id: 238
relations: ["INSIDE-['kitchen']"]
obj_name: desk, id: 110
relations: ["INSIDE-['bedroom']", "ON-['floor']"]
obj_name: fridge, id: 306
relations: ["INSIDE-['kitchen']", "ON-['floor']"]
obj_name: bookshelf, id: 107
relations: ["INSIDE-['bedroom']", "ON-['floor']"]
obj_name: stove, id: 312
relations: ["INSIDE-['kitchen']"]


## **8. Check the Relationship Between Characters and Objects**  

Before a character can perform an action on an object, it is essential to check whether the relationship between the character and the object satisfies the action's requirements. This can be done by inspecting the `environment_graph`. If a character is near or holding an object, the relevant relationship information will be stored in the graph.  
This script checks if the character has acquired the `salmon` and outputs the last five relationships of the character, which should confirm the interaction and provide insights into the relationship between the character and objects.

### **Code Explanation**  
- **Reset Environment**:  
  - The environment is reset with `comm.reset(0)` and the updated graph is retrieved.  

- **Add Character**:  
  - A character (`Male1`) is added to the environment using `comm.add_character('Chars/Male1')`.  

- **Get Object ID**:  
  - The ID of the `salmon` object is retrieved from the graph.  

- **Define Script**:  
  - The script defines a sequence of actions where the character walks to and grabs the `salmon`.  

- **Execute Script**:  
  - The `comm.render_script(scripts, recording=True, skip_animation=True)` method is used to render the actions, without displaying the animation but recording the result.  

- **Check Character Relationships**:  
  - The graph is examined again, and the relationships of all characters are printed. This helps confirm that the character has the proper relationship with the object (e.g., holding or near the `salmon`).  


In [11]:
res = comm.reset(0)
res, g = comm.environment_graph()
comm.add_character('Chars/Male1')
salmon_id = [node['id'] for node in g['nodes'] if node['class_name'] == 'salmon'][0]
print(f'salmon_id: {salmon_id}')
scripts = [
    f'<char0> [walk] <salmon> ({salmon_id})',
    f'<char0> [grab] <salmon> ({salmon_id})'
]
comm.render_script(scripts, recording=True, skip_animation=True)
res, g = comm.environment_graph()
character_list = [node for node in g['nodes'] if node['class_name'] == 'character']
for character in character_list:
    rels = [edge for edge in g['edges'] if edge['from_id'] == character['id']]
    pprint.pprint(rels[-5:])


salmon_id: 328
[{'from_id': 1, 'relation_type': 'CLOSE', 'to_id': 311},
 {'from_id': 1, 'relation_type': 'CLOSE', 'to_id': 45},
 {'from_id': 1, 'relation_type': 'CLOSE', 'to_id': 235},
 {'from_id': 1, 'relation_type': 'CLOSE', 'to_id': 266},
 {'from_id': 1, 'relation_type': 'HOLDS_RH', 'to_id': 328}]


**Relation Type 说明**

| Relation Type | Meaning                                                                 |
|---------------|-------------------------------------------------------------------------|
| ON            | Object `from_id` is on top of object `to_id`.                           |
| INSIDE        | Object `from_id` is inside of object `to_id`.                           |
| BETWEEN       | Used for doors. Door connects with room `to_id`.                        |
| CLOSE         | Object `from_id` is close to object `to_id` (< 1.5 metres).             |
| FACING        | Object `to_id` is visible from object `from_id` and distance is < 5 metres. If `from_id` is a sofa or a chair, it should also be turned towards `to_id`. |
| HOLDS_RH      | Character `from_id` holds object `to_id` with the right hand.           |
| HOLDS_LH      | Character `from_id` holds object `to_id` with the left hand.            |
| SITTING       | Character `from_id` is sitting in object `to_id`.                       |

 

  
**Action 说明**
| Action        | Arguments | Modifier | Preconditions                                                                 | Postconditions                                                                 | Example                     |
|---------------|-----------|----------|-------------------------------------------------------------------------------|--------------------------------------------------------------------------------|-----------------------------|
| **Walk**      | 1         |          | - Character is not sitting.<br>- `obj1` is reachable or is a room.<br>- `obj1` is not grabbed. | - Character moves close to `obj1`, ensuring visibility.<br>- If `obj1` is a room, character moves to its center. | `[walk] <kitchen> (1)`      |
| **Run**       | 1         |          | - Character is not sitting.<br>- `obj1` is reachable or is a room.<br>- `obj1` is not grabbed. | - Character moves close to `obj1`, ensuring visibility.<br>- If `obj1` is a room, character moves to its center. | `[run] <kitchen> (1)`       |
| **Walktowards** | 1         |          | - Character is not sitting.<br>- `obj1` is reachable or is a room.<br>- `obj1` is not grabbed. | - Character moves close to `obj1`, ensuring visibility.<br>- If `obj1` is a room, character moves to its center. | `[walktowards] <kitchen> (1)`<br>`[walktowards] <table> (1) :3:` |
| **Walkforward** | 0         |          | - Character is not sitting.                                                   | - Character moves 1 meter forward according to current orientation.            | `[walkforward]`             |
| **TurnLeft**  | 0         |          | - Character is not sitting.                                                   | - Character turns 30 degrees counterclockwise.                                 | `[turnleft]`                |
| **TurnRight** | 0         |          | - Character is not sitting.                                                   | - Character turns 30 degrees clockwise.                                        | `[turnright]`               |
| **Sit**       | 1         |          | - Character is not sitting.<br>- Character is close to `obj1`.<br>- `obj1` has property `sittable`.<br>- Number of objects on `obj1` must not exceed capacity. | - Adds directed edge: `character sitting obj1`.                                | `[sit] <chair> (1)`         |
| **StandUp**   | 0         |          | - Character is sitting.                                                       | - Removes `sitting` state from character.                                      | `[standup]`                 |
| **Grab**      | 1         |          | - `obj1` is grabbable (except water).<br>- Character is close to `obj1`.<br>- `obj1` is reachable.<br>- Character has at least one free hand. | - Adds directed edge: `character holds_rh` or `holds_lh obj1`.<br>- `obj1` is no longer on a surface or inside a container. | `[grab] <apple> (1)`        |
| **Open**      | 1         |          | - `obj1` is opennable and closed.<br>- Character is close to `obj1`.<br>- `obj1` is reachable.<br>- Character has at least one free hand. | - `obj1` state changes to `open`.                                              | `[open] <fridge> (1)`       |
| **Close**     | 1         |          | - `obj1` is opennable and open.<br>- Character is close to `obj1`.<br>- `obj1` is reachable.<br>- Character has at least one free hand. | - `obj1` state changes to `closed`.                                            | `[close] <fridge> (1)`      |
| **Put**       | 2         |          | - Character holds `obj1` (left or right hand).<br>- Character is close to `obj2`. | - Removes directed edge: `character holds_lh` or `holds_rh obj1`.<br>- Adds directed edge: `obj1 on obj2`. | `[putback] <apple> (1) <table> (1)` |
| **PutIn**     | 2         |          | - Character holds `obj1` (left or right hand).<br>- Character is close to `obj2`.<br>- `obj2` is not closed. | - Removes directed edge: `character holds_lh` or `holds_rh obj1`.<br>- Adds directed edge: `obj1 inside obj2`. | `[putin] <apple> (1) <fridge> (1)` |
| **SwitchOn**  | 1         |          | - `obj1` has property `switch` and is off.<br>- Character is close to `obj1`. | - `obj1` state changes to `on`.                                                | `[switchon] <stove> (1)`    |
| **SwitchOff** | 1         |          | - `obj1` has property `switch` and is on.<br>- Character is close to `obj1`.  | - `obj1` state changes to `off`.                                               | `[switchoff] <stove> (1)`   |
| **Drink**     | 1         |          | - `obj1` is drinkable or a recipient.<br>- Character is close to `obj1`.      | -                                                                              | `[drink] <waterglass> (1)`  |
| **Touch**     | 1         |          | - Character is close to `obj1`.<br>- `obj1` is reachable.                     | -                                                                              | `[touch] <stove> (1)`       |
| **LookAt**    | 1         |          | - Character is facing `obj1`.<br>- `obj1` is reachable.                       | -                                                                              | `[lookat] <stove> (1)`      |